<a href="https://colab.research.google.com/github/estebanmartinez5/Kaggle_Saber_Pro_Colombia/blob/main/03_modelo_con_preprocesado_inicial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"danielloaizacarmona","key":"cf9d9029f8c19118e9f0c7abab4bac38"}'}

In [98]:
import os
import zipfile

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [99]:

!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [100]:
with zipfile.ZipFile("udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip", "r") as zip_ref:
    zip_ref.extractall("datos")

In [105]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer



data = pd.read_csv("datos/train.csv")

El objetivo principal de este análisis y preprocesamiento de datos es preparar el conjunto de datos para su posterior uso en la creación de un modelo predictivo. Esto implica la identificaión y eliminación de columnas redundantes, el análisis de la distribucion de valores en las diferentes columnas, el convertir columnas en OneHot, la conversion de valores "Si/No" a "True/False", entre otros procesos de limpieza. El resultado final es un conjunto de datos limpio y transformado, listo para su análisis y modelado.


In [102]:
print(data.columns)

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL',
       'coef_1', 'coef_2', 'coef_3', 'coef_4'],
      dtype='object')


In [96]:
missing_values = data.isnull().sum()

print("Valores faltantes por columna:")
print(missing_values)

Valores faltantes por columna:
ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_EDUCACIONPADRE               23178
FAMI_TIENELAVADORA                39773
FAMI_TIENEAUTOMOVIL               43623
ESTU_PRIVADO_LIBERTAD                 0
ESTU_PAGOMATRICULAPROPIO           6498
FAMI_TIENECOMPUTADOR              38103
FAMI_TIENEINTERNET.1              26629
FAMI_EDUCACIONMADRE               23664
RENDIMIENTO_GLOBAL                    0
dtype: int64




  # *Eliminación de la columna "FAMI_TIENEINTERNET"*
  # *Eliminacion de las columna "coef_"







Ya que la columna "FAMI_TIENEINTERNET" está duplicada, se decidío eliminarla, ya que una de ellas no aportaba información diferente.
Por otro lado se eliminan las columnas coef_ ya que al parecer son informacion compacatda de las demas columnas


In [106]:

columnas_redundantes = ['FAMI_TIENEINTERNET']

data = data.drop(columnas_redundantes, axis=1)
data = data.drop(columns=['coef_1'])
data = data.drop(columns=['coef_2'])
data = data.drop(columns=['coef_3'])
data = data.drop(columns=['coef_4'])

print(data.columns)

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA',
       'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL'],
      dtype='object')


#  Análisis de distribución de valores por columna


In [107]:

columnas_excluir = ['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
                    'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
                    'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE',
                    'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL']

for columna in data.columns:
    if columna not in columnas_excluir:

        conteo_valores = data[columna].value_counts()

        print(f"Valores de la columna '{columna}':")
        print(conteo_valores)
        print()


Valores de la columna 'FAMI_TIENELAVADORA':
FAMI_TIENELAVADORA
Si    563390
No     89337
Name: count, dtype: int64

Valores de la columna 'FAMI_TIENEAUTOMOVIL':
FAMI_TIENEAUTOMOVIL
No    412606
Si    236271
Name: count, dtype: int64

Valores de la columna 'ESTU_PRIVADO_LIBERTAD':
ESTU_PRIVADO_LIBERTAD
N    692466
S        34
Name: count, dtype: int64

Valores de la columna 'ESTU_PAGOMATRICULAPROPIO':
ESTU_PAGOMATRICULAPROPIO
No    382201
Si    303801
Name: count, dtype: int64

Valores de la columna 'FAMI_TIENECOMPUTADOR':
FAMI_TIENECOMPUTADOR
Si    597670
No     56727
Name: count, dtype: int64

Valores de la columna 'FAMI_TIENEINTERNET.1':
FAMI_TIENEINTERNET.1
Si    592514
No     73357
Name: count, dtype: int64



# *Eliminación de la columna "ESTU_PRIVADO_LIBERTAD"*


Ya que todos los valores de la columna "ESTU_PRIVADO_LIBERTAD" son "No", se decidío eliminarla, ya que esto sugiere que esta columna no desempeña un papel significativo en el modelo.

In [108]:

data.drop('ESTU_PRIVADO_LIBERTAD', axis=1, inplace=True)

print(data.columns)

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA',
       'FAMI_TIENEAUTOMOVIL', 'ESTU_PAGOMATRICULAPROPIO',
       'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE',
       'RENDIMIENTO_GLOBAL'],
      dtype='object')


Ya que las columnas "ESTU_VALORMATRICULAUNIVERSIDAD" y
"ESTU_HORASSEMANATRABAJA" contienen  rangos en sus valores, se decidió convertirlas en one-hot.


In [109]:


valormatricula_count = data['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

horas_trabajo_count = data['ESTU_HORASSEMANATRABAJA'].value_counts()

print("Recuento de valores únicos en 'ESTU_VALORMATRICULAUNIVERSIDAD':")
print(valormatricula_count)
print("\nRecuento de valores únicos en 'ESTU_HORASSEMANATRABAJA':")
print(horas_trabajo_count)


Recuento de valores únicos en 'ESTU_VALORMATRICULAUNIVERSIDAD':
ESTU_VALORMATRICULAUNIVERSIDAD
Entre 1 millón y menos de 2.5 millones      204048
Entre 2.5 millones y menos de 4 millones    127430
Menos de 500 mil                             80263
Entre 500 mil y menos de 1 millón            78704
Entre 4 millones y menos de 5.5 millones     69736
Más de 7 millones                            68014
Entre 5.5 millones y menos de 7 millones     38490
No pagó matrícula                            19528
Name: count, dtype: int64

Recuento de valores únicos en 'ESTU_HORASSEMANATRABAJA':
ESTU_HORASSEMANATRABAJA
Más de 30 horas        249352
0                      116550
Entre 11 y 20 horas    115857
Entre 21 y 30 horas     92693
Menos de 10 horas       87191
Name: count, dtype: int64


In [110]:

valormatricula_dummies = pd.get_dummies(data['ESTU_VALORMATRICULAUNIVERSIDAD'], prefix='VALOR_MATRICULA')
data = pd.concat([data, valormatricula_dummies], axis=1)
data.drop('ESTU_VALORMATRICULAUNIVERSIDAD', axis=1, inplace=True)

horassemana_dummies = pd.get_dummies(data['ESTU_HORASSEMANATRABAJA'], prefix='HORAS_TRABAJO_SEMANAL')
data = pd.concat([data, horassemana_dummies], axis=1)
data.drop('ESTU_HORASSEMANATRABAJA', axis=1, inplace=True)

print(data)


            ID  PERIODO                  ESTU_PRGM_ACADEMICO  \
0       904256    20212                           ENFERMERIA   
1       645256    20212                              DERECHO   
2       308367    20203                MERCADEO Y PUBLICIDAD   
3       470353    20195           ADMINISTRACION DE EMPRESAS   
4       989032    20212                           PSICOLOGIA   
...        ...      ...                                  ...   
692495   25096    20195                             BIOLOGIA   
692496  754213    20212                           PSICOLOGIA   
692497  504185    20183  ADMINISTRACIÓN EN SALUD OCUPACIONAL   
692498  986620    20195                           PSICOLOGIA   
692499  933374    20195                           PSICOLOGIA   

       ESTU_PRGM_DEPARTAMENTO FAMI_ESTRATOVIVIENDA  \
0                      BOGOTÁ            Estrato 3   
1                   ATLANTICO            Estrato 3   
2                      BOGOTÁ            Estrato 3   
3              

Columnas con valores "Si/No"

In [111]:

columnas_si_no = []

for columna in data.columns:

    valores_unicos = data[columna].unique()

    if 'Si' in valores_unicos and 'No' in valores_unicos:
        columnas_si_no.append(columna)

print("Columnas con valores 'Si' y 'No':")
for columna in columnas_si_no:
    print(columna)


Columnas con valores 'Si' y 'No':
FAMI_TIENELAVADORA
FAMI_TIENEAUTOMOVIL
ESTU_PAGOMATRICULAPROPIO
FAMI_TIENECOMPUTADOR
FAMI_TIENEINTERNET.1


In [112]:

data = data.applymap(lambda x: True if x == 'Si' else (False if x == 'No' else x))

print(data)

/tmp/ipython-input-112-1327736958.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: True if x == 'Si' else (False if x == 'No' else x))


            ID  PERIODO                  ESTU_PRGM_ACADEMICO  \
0       904256    20212                           ENFERMERIA   
1       645256    20212                              DERECHO   
2       308367    20203                MERCADEO Y PUBLICIDAD   
3       470353    20195           ADMINISTRACION DE EMPRESAS   
4       989032    20212                           PSICOLOGIA   
...        ...      ...                                  ...   
692495   25096    20195                             BIOLOGIA   
692496  754213    20212                           PSICOLOGIA   
692497  504185    20183  ADMINISTRACIÓN EN SALUD OCUPACIONAL   
692498  986620    20195                           PSICOLOGIA   
692499  933374    20195                           PSICOLOGIA   

       ESTU_PRGM_DEPARTAMENTO FAMI_ESTRATOVIVIENDA  \
0                      BOGOTÁ            Estrato 3   
1                   ATLANTICO            Estrato 3   
2                      BOGOTÁ            Estrato 3   
3              

In [113]:

print("Distribución original de los valores de rendimiento:")
print(data['RENDIMIENTO_GLOBAL'].value_counts())


Distribución original de los valores de rendimiento:
RENDIMIENTO_GLOBAL
alto          175619
bajo          172987
medio-bajo    172275
medio-alto    171619
Name: count, dtype: int64


In [114]:

mapeo_rangos = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}


data['RENDIMIENTO_GLOBAL'] = data['RENDIMIENTO_GLOBAL'].map(mapeo_rangos)

print("\nDistribución después de la codificación ordinal:")
print(data['RENDIMIENTO_GLOBAL'].value_counts())


Distribución después de la codificación ordinal:
RENDIMIENTO_GLOBAL
3    175619
0    172987
1    172275
2    171619
Name: count, dtype: int64


In [115]:

print(data['FAMI_ESTRATOVIVIENDA'].head(5))

data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].str.replace('Estrato ', '')

print(data['FAMI_ESTRATOVIVIENDA'].head(5))


0    Estrato 3
1    Estrato 3
2    Estrato 3
3    Estrato 4
4    Estrato 3
Name: FAMI_ESTRATOVIVIENDA, dtype: object
0    3
1    3
2    3
3    4
4    3
Name: FAMI_ESTRATOVIVIENDA, dtype: object


In [116]:

print("\nTodos los registros del DataFrame:")
print(data)



Todos los registros del DataFrame:
            ID  PERIODO                  ESTU_PRGM_ACADEMICO  \
0       904256    20212                           ENFERMERIA   
1       645256    20212                              DERECHO   
2       308367    20203                MERCADEO Y PUBLICIDAD   
3       470353    20195           ADMINISTRACION DE EMPRESAS   
4       989032    20212                           PSICOLOGIA   
...        ...      ...                                  ...   
692495   25096    20195                             BIOLOGIA   
692496  754213    20212                           PSICOLOGIA   
692497  504185    20183  ADMINISTRACIÓN EN SALUD OCUPACIONAL   
692498  986620    20195                           PSICOLOGIA   
692499  933374    20195                           PSICOLOGIA   

       ESTU_PRGM_DEPARTAMENTO FAMI_ESTRATOVIVIENDA  \
0                      BOGOTÁ                    3   
1                   ATLANTICO                    3   
2                      BOGOTÁ    

In [117]:

for columna in data.columns:

    conteo_valores = data[columna].value_counts()

    print(f"Valores de la columna '{columna}':")
    print(conteo_valores)
    print()


Valores de la columna 'ID':
ID
933374    1
904256    1
645256    1
308367    1
706675    1
         ..
257869    1
11829     1
47159     1
659872    1
989032    1
Name: count, Length: 692500, dtype: int64

Valores de la columna 'PERIODO':
PERIODO
20195    180873
20203    171838
20212    171412
20183    164818
20194      1472
20213      1178
20202       490
20184       254
20196       165
Name: count, dtype: int64

Valores de la columna 'ESTU_PRGM_ACADEMICO':
ESTU_PRGM_ACADEMICO
DERECHO                                                   53244
ADMINISTRACION DE EMPRESAS                                51902
CONTADURIA PUBLICA                                        39664
PSICOLOGIA                                                31932
INGENIERIA INDUSTRIAL                                     28481
                                                          ...  
DEPORTE Y ACTIVIDADA FISICA                                   1
QUIMICA Y FARMACIA                                            1
LICEN

In [118]:

for columna in data.columns:

    print(f"Valores únicos en la columna '{columna}':")

    print(data[columna].unique())
    print()


Valores únicos en la columna 'ID':
[904256 645256 308367 ... 504185 986620 933374]

Valores únicos en la columna 'PERIODO':
[20212 20203 20195 20183 20194 20213 20184 20202 20196]

Valores únicos en la columna 'ESTU_PRGM_ACADEMICO':
['ENFERMERIA' 'DERECHO' 'MERCADEO Y PUBLICIDAD'
 'ADMINISTRACION DE EMPRESAS' 'PSICOLOGIA' 'MEDICINA VETERINARIA'
 'INGENIERIA MECANICA' 'ADMINISTRACIÓN EN SALUD OCUPACIONAL'
 'INGENIERIA INDUSTRIAL' 'ADMINISTRACIÓN FINANCIERA' 'HOTELERIA Y TURISMO'
 'LICENCIATURA EN CIENCIAS SOCIALES' 'LICENCIATURA EN PEDAGOGIA INFANTIL'
 'COMUNICACION SOCIAL' 'CIENCIA POLITICA'
 'PROFESIONAL EN GESTIÓN DE LA SEGURIDAD Y LA SALUD LABORAL'
 'MAESTRO EN MÚSICA' 'INGENIERIA MECATRONICA' 'TRABAJO SOCIAL'
 'LICENCIATURA EN BIOLOGIA Y EDUCACION AMBIENTAL' 'INGENIERIA CIVIL'
 'CONTADURIA PÚBLICA' 'ADMINISTRACION EN SALUD'
 'ADMINISTRACIÓN DE EMPRESAS' 'ESTADISTICA' 'LICENCIATURA EN BIOLOGIA'
 'INGENIERIA AGROINDUSTRIAL' 'ZOOTECNIA' 'COMUNICACION AUDIOVISUAL'
 'LICENCIATURA EN EDU

In [119]:

for columna in data.columns:

    suma_nulos = data[columna].isnull().sum()

    print(f"Columna '{columna}': {suma_nulos} valores nulos")


Columna 'ID': 0 valores nulos
Columna 'PERIODO': 0 valores nulos
Columna 'ESTU_PRGM_ACADEMICO': 0 valores nulos
Columna 'ESTU_PRGM_DEPARTAMENTO': 0 valores nulos
Columna 'FAMI_ESTRATOVIVIENDA': 32137 valores nulos
Columna 'FAMI_EDUCACIONPADRE': 23178 valores nulos
Columna 'FAMI_TIENELAVADORA': 39773 valores nulos
Columna 'FAMI_TIENEAUTOMOVIL': 43623 valores nulos
Columna 'ESTU_PAGOMATRICULAPROPIO': 6498 valores nulos
Columna 'FAMI_TIENECOMPUTADOR': 38103 valores nulos
Columna 'FAMI_TIENEINTERNET.1': 26629 valores nulos
Columna 'FAMI_EDUCACIONMADRE': 23664 valores nulos
Columna 'RENDIMIENTO_GLOBAL': 0 valores nulos
Columna 'VALOR_MATRICULA_Entre 1 millón y menos de 2.5 millones': 0 valores nulos
Columna 'VALOR_MATRICULA_Entre 2.5 millones y menos de 4 millones': 0 valores nulos
Columna 'VALOR_MATRICULA_Entre 4 millones y menos de 5.5 millones': 0 valores nulos
Columna 'VALOR_MATRICULA_Entre 5.5 millones y menos de 7 millones': 0 valores nulos
Columna 'VALOR_MATRICULA_Entre 500 mil y men

In [120]:
madres = data['FAMI_EDUCACIONMADRE'].value_counts()
padres = data['FAMI_EDUCACIONPADRE'].value_counts()

print("Educación de las madres:")
print(madres)
print("---------------")
print("Educación de los padres:")
print(padres)


Educación de las madres:
FAMI_EDUCACIONMADRE
Secundaria (Bachillerato) completa      141744
Primaria incompleta                      99420
Técnica o tecnológica completa           89542
Educación profesional completa           85326
Secundaria (Bachillerato) incompleta     81012
Primaria completa                        56125
Postgrado                                46246
Técnica o tecnológica incompleta         27533
Educación profesional incompleta         22470
Ninguno                                  14483
No sabe                                   3017
No Aplica                                 1918
Name: count, dtype: int64
---------------
Educación de los padres:
FAMI_EDUCACIONPADRE
Secundaria (Bachillerato) completa      128289
Primaria incompleta                     125675
Educación profesional completa           83117
Secundaria (Bachillerato) incompleta     71654
Técnica o tecnológica completa           62995
Primaria completa                        55958
Postgrado             

In [121]:

moda_estrato = data['FAMI_ESTRATOVIVIENDA'].mode()[0]

data['FAMI_ESTRATOVIVIENDA'].fillna(moda_estrato, inplace=True)

/tmp/ipython-input-121-2679772169.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['FAMI_ESTRATOVIVIENDA'].fillna(moda_estrato, inplace=True)


In [122]:

educacion_padre_counts = data['FAMI_EDUCACIONPADRE'].value_counts()
print(educacion_padre_counts)
print("====================================")


educacion_madre_counts = data['FAMI_EDUCACIONMADRE'].value_counts()
print(educacion_madre_counts)


moda_padre = data['FAMI_EDUCACIONPADRE'].mode()[0]
data['FAMI_EDUCACIONPADRE'].fillna(moda_padre, inplace=True)


moda_madre = data['FAMI_EDUCACIONMADRE'].mode()[0]
data['FAMI_EDUCACIONMADRE'].fillna(moda_madre, inplace=True)

FAMI_EDUCACIONPADRE
Secundaria (Bachillerato) completa      128289
Primaria incompleta                     125675
Educación profesional completa           83117
Secundaria (Bachillerato) incompleta     71654
Técnica o tecnológica completa           62995
Primaria completa                        55958
Postgrado                                44169
Educación profesional incompleta         27084
Técnica o tecnológica incompleta         22552
Ninguno                                  22008
No sabe                                  16592
No Aplica                                 9229
Name: count, dtype: int64
FAMI_EDUCACIONMADRE
Secundaria (Bachillerato) completa      141744
Primaria incompleta                      99420
Técnica o tecnológica completa           89542
Educación profesional completa           85326
Secundaria (Bachillerato) incompleta     81012
Primaria completa                        56125
Postgrado                                46246
Técnica o tecnológica incompleta         

/tmp/ipython-input-122-3343380376.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['FAMI_EDUCACIONPADRE'].fillna(moda_padre, inplace=True)
/tmp/ipython-input-122-3343380376.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

In [123]:
columnas_a_eliminar = ['FAMI_TIENELAVADORA', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'FAMI_TIENEAUTOMOVIL']


data = data.drop(columnas_a_eliminar, axis=1)


print(data.head())

       ID  PERIODO FAMI_ESTRATOVIVIENDA                 FAMI_EDUCACIONPADRE  \
0  904256    20212                    3    Técnica o tecnológica incompleta   
1  645256    20212                    3      Técnica o tecnológica completa   
2  308367    20203                    3  Secundaria (Bachillerato) completa   
3  470353    20195                    4                             No sabe   
4  989032    20212                    3                   Primaria completa   

  ESTU_PAGOMATRICULAPROPIO FAMI_TIENECOMPUTADOR FAMI_TIENEINTERNET.1  \
0                    False                 True                 True   
1                    False                 True                False   
2                    False                False                 True   
3                    False                 True                 True   
4                    False                 True                 True   

                  FAMI_EDUCACIONMADRE  RENDIMIENTO_GLOBAL  \
0                           Pos

In [124]:
print("Datos faltantes por columna:")
print(data.isnull().sum())

Datos faltantes por columna:
ID                                                              0
PERIODO                                                         0
FAMI_ESTRATOVIVIENDA                                            0
FAMI_EDUCACIONPADRE                                             0
ESTU_PAGOMATRICULAPROPIO                                     6498
FAMI_TIENECOMPUTADOR                                        38103
FAMI_TIENEINTERNET.1                                        26629
FAMI_EDUCACIONMADRE                                             0
RENDIMIENTO_GLOBAL                                              0
VALOR_MATRICULA_Entre 1 millón y menos de 2.5 millones          0
VALOR_MATRICULA_Entre 2.5 millones y menos de 4 millones        0
VALOR_MATRICULA_Entre 4 millones y menos de 5.5 millones        0
VALOR_MATRICULA_Entre 5.5 millones y menos de 7 millones        0
VALOR_MATRICULA_Entre 500 mil y menos de 1 millón               0
VALOR_MATRICULA_Menos de 500 mil               

In [125]:

columnas_booleanas = ['FAMI_TIENEINTERNET.1',
                      'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR']


for columna in columnas_booleanas:

    moda = data[columna].mode()[0]

    data[columna].fillna(moda, inplace=True)


print(data.head())

/tmp/ipython-input-125-1009508335.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[columna].fillna(moda, inplace=True)
/tmp/ipython-input-125-1009508335.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[columna].fillna(moda, inplace=True)


       ID  PERIODO FAMI_ESTRATOVIVIENDA                 FAMI_EDUCACIONPADRE  \
0  904256    20212                    3    Técnica o tecnológica incompleta   
1  645256    20212                    3      Técnica o tecnológica completa   
2  308367    20203                    3  Secundaria (Bachillerato) completa   
3  470353    20195                    4                             No sabe   
4  989032    20212                    3                   Primaria completa   

   ESTU_PAGOMATRICULAPROPIO  FAMI_TIENECOMPUTADOR  FAMI_TIENEINTERNET.1  \
0                     False                  True                  True   
1                     False                  True                 False   
2                     False                 False                  True   
3                     False                  True                  True   
4                     False                  True                  True   

                  FAMI_EDUCACIONMADRE  RENDIMIENTO_GLOBAL  \
0            

In [126]:
print("Datos faltantes por columna:")
print(data.isnull().sum())

Datos faltantes por columna:
ID                                                          0
PERIODO                                                     0
FAMI_ESTRATOVIVIENDA                                        0
FAMI_EDUCACIONPADRE                                         0
ESTU_PAGOMATRICULAPROPIO                                    0
FAMI_TIENECOMPUTADOR                                        0
FAMI_TIENEINTERNET.1                                        0
FAMI_EDUCACIONMADRE                                         0
RENDIMIENTO_GLOBAL                                          0
VALOR_MATRICULA_Entre 1 millón y menos de 2.5 millones      0
VALOR_MATRICULA_Entre 2.5 millones y menos de 4 millones    0
VALOR_MATRICULA_Entre 4 millones y menos de 5.5 millones    0
VALOR_MATRICULA_Entre 5.5 millones y menos de 7 millones    0
VALOR_MATRICULA_Entre 500 mil y menos de 1 millón           0
VALOR_MATRICULA_Menos de 500 mil                            0
VALOR_MATRICULA_Más de 7 millones        

In [127]:

print(data.columns)


num_columnas = len(data.columns)
print("Número de columnas:", num_columnas)

Index(['ID', 'PERIODO', 'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL',
       'VALOR_MATRICULA_Entre 1 millón y menos de 2.5 millones',
       'VALOR_MATRICULA_Entre 2.5 millones y menos de 4 millones',
       'VALOR_MATRICULA_Entre 4 millones y menos de 5.5 millones',
       'VALOR_MATRICULA_Entre 5.5 millones y menos de 7 millones',
       'VALOR_MATRICULA_Entre 500 mil y menos de 1 millón',
       'VALOR_MATRICULA_Menos de 500 mil', 'VALOR_MATRICULA_Más de 7 millones',
       'VALOR_MATRICULA_No pagó matrícula', 'HORAS_TRABAJO_SEMANAL_0',
       'HORAS_TRABAJO_SEMANAL_Entre 11 y 20 horas',
       'HORAS_TRABAJO_SEMANAL_Entre 21 y 30 horas',
       'HORAS_TRABAJO_SEMANAL_Menos de 10 horas',
       'HORAS_TRABAJO_SEMANAL_Más de 30 horas'],
      dtype='object')
Número de columnas: 22


In [128]:
data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].replace('Sin Estrato', '0')
data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].str.replace('Estrato ', '').astype(int)


data = pd.get_dummies(data)

X = data.drop('RENDIMIENTO_GLOBAL', axis=1)
y = data['RENDIMIENTO_GLOBAL']


from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [129]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy en el conjunto de validación: {accuracy}')



Accuracy en el conjunto de validación: 0.32726353790613716


In [130]:
import pandas as pd

test_data = pd.read_csv("datos/train.csv")


test_data['FAMI_ESTRATOVIVIENDA'] = test_data['FAMI_ESTRATOVIVIENDA'].replace('Sin Estrato', '0')


test_data['FAMI_ESTRATOVIVIENDA'] = test_data['FAMI_ESTRATOVIVIENDA'].str.replace('Estrato ', '')


test_data['FAMI_ESTRATOVIVIENDA'] = test_data['FAMI_ESTRATOVIVIENDA'].fillna('0').astype(int)


test_data = pd.get_dummies(test_data)


missing_cols = set(X.columns) - set(test_data.columns)
for col in missing_cols:
    test_data[col] = 0
test_data = test_data[X.columns]


test_predictions = model.predict(test_data)


submission = pd.DataFrame({'ID': test_data.index, 'RENDIMIENTO_GLOBAL': test_predictions})
submission.to_csv('submission.csv', index=False)
print("Predicciones guardadas en submission.csv")

Predicciones guardadas en submission.csv
